In [1]:
import requests
import json
import os
from pprint import pprint
from pymongo import MongoClient
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [2]:
client = MongoClient('localhost', 27017)

# Select your database
db = client['nyc']

# Select your collection
collection = db['rats']

In [3]:
fields_to_check = [
    "INSPECTION_TYPE",
    "ZIP_CODE",
    "BOROUGH",
    "INSPECTION_DATE",
    "RESULT"
]

query = {"$or": []}
for field in fields_to_check:
    query["$or"].extend([
        {field: {"$exists": False}},
        {field: ""},
        {field: {"$regex": "^\s*$"}}
    ])
query["$or"].append({"ZIP_CODE": 0})

coord_fields_to_check = ["LATITUDE", 'LONGITUDE']

for field in coord_fields_to_check:
    query["$or"].extend([
        {field: {"$exists": False}},
        {field: ""},
        {field: {"$regex": "^\s*$"}},
        {field: 0}
    ])

result = collection.delete_many(query)

print(f"Deleted {result.deleted_count} documents.")

Deleted 32313 documents.


In [4]:
#Remove all documents with an Inspection Date before 2023
# Define the date threshold
threshold_date = datetime(2023, 1, 1)

# Remove documents with an INSPECTION_DATE before 2023
result = collection.delete_many({
    "INSPECTION_DATE": {
        "$lt": threshold_date
    }
})

print(f"{result.deleted_count} documents were deleted.")


2246248 documents were deleted.


In [1]:
#Bring data in from Mongo to Pandas
cursor = collection.find({})
df = pd.DataFrame(list(cursor))

#Drop _id column
df.drop(columns=['_id'], inplace=True) 

NameError: name 'collection' is not defined

In [10]:
#View df to check everything looks good
df

,INSPECTION_TYPE,ZIP_CODE,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT
0,Initial,12345,40.817678,-73.941974,Manhattan,2023-03-08 15:21:41,Passed
1,Initial,11377,40.738373,-73.906470,Queens,2023-06-05 20:19:22,Passed
2,Initial,10457,40.850038,-73.894424,Bronx,2023-07-17 16:05:21,Passed
3,BAIT,11385,40.708495,-73.919696,Queens,2023-04-20 17:18:23,Bait applied
4,Initial,10470,40.897316,-73.863219,Bronx,2023-03-14 13:40:50,Failed for Other R
...,...,...,...,...,...,...,...
162372,Initial,10065,40.764009,-73.966893,Manhattan,2023-02-03 16:55:09,Passed
162373,Initial,10458,40.856980,-73.886359,Bronx,2023-02-03 20:30:05,Passed
162374,Compliance,11211,40.707009,-73.951506,Brooklyn,2023-05-26 17:10:32,Rat Activity
162375,Initial,11206,40.694630,-73.935954,Brooklyn,2023-02-03 21:00:12,Rat Activity


In [11]:
#RAT ACTIVITY IS THE TARGET OF OUR ML MODEL
# Create a new column "Rat Activity" and initialize with 0
df['RAT_ACTIVITY'] = 0

# Set the "Rat_Activity" column to 1 where there is rat activity
df.loc[(df['INSPECTION_TYPE'] == 'Initial') & (df['RESULT'] == 'Rat Activity'), 'RAT_ACTIVITY'] = 1
df.loc[(df['INSPECTION_TYPE'] == 'Compliance') & (df['RESULT'] == 'Rat Activity'), 'RAT_ACTIVITY'] = 1
df.loc[df['INSPECTION_TYPE'].isin(['BAIT', 'STOPPAGE', 'CLEAN_UPS']), 'RAT_ACTIVITY'] = 1

df


,INSPECTION_TYPE,ZIP_CODE,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,RAT_ACTIVITY
0,Initial,12345,40.817678,-73.941974,Manhattan,2023-03-08 15:21:41,Passed,0
1,Initial,11377,40.738373,-73.906470,Queens,2023-06-05 20:19:22,Passed,0
2,Initial,10457,40.850038,-73.894424,Bronx,2023-07-17 16:05:21,Passed,0
3,BAIT,11385,40.708495,-73.919696,Queens,2023-04-20 17:18:23,Bait applied,1
4,Initial,10470,40.897316,-73.863219,Bronx,2023-03-14 13:40:50,Failed for Other R,0
...,...,...,...,...,...,...,...,...
162372,Initial,10065,40.764009,-73.966893,Manhattan,2023-02-03 16:55:09,Passed,0
162373,Initial,10458,40.856980,-73.886359,Bronx,2023-02-03 20:30:05,Passed,0
162374,Compliance,11211,40.707009,-73.951506,Brooklyn,2023-05-26 17:10:32,Rat Activity,1
162375,Initial,11206,40.694630,-73.935954,Brooklyn,2023-02-03 21:00:12,Rat Activity,1


In [12]:
#Data engineering features
# Convert the 'INSPECTION_DATE' column to datetime format
df['INSPECTION_DATE'] = pd.to_datetime(df['INSPECTION_DATE'])

# Extract the month from the 'INSPECTION_DATE' column
df['INSPECTION_MONTH'] = df['INSPECTION_DATE'].dt.month

# Print the updated dataframe
df[['INSPECTION_DATE', 'INSPECTION_MONTH']]

,INSPECTION_DATE,INSPECTION_MONTH
0,2023-03-08 15:21:41,3
1,2023-06-05 20:19:22,6
2,2023-07-17 16:05:21,7
3,2023-04-20 17:18:23,4
4,2023-03-14 13:40:50,3
...,...,...
162372,2023-02-03 16:55:09,2
162373,2023-02-03 20:30:05,2
162374,2023-05-26 17:10:32,5
162375,2023-02-03 21:00:12,2


In [13]:
# Group the data by ZIP code and compute the mean of RAT_ACTIVITY
#mean encode zip code
mean_rat_activity = df.groupby('ZIP_CODE')['RAT_ACTIVITY'].mean()

# Create a dictionary mapping each ZIP code to its corresponding mean
zip_code_to_mean = dict(mean_rat_activity)

# Replace the values in the ZIP_CODE column with the values from the dictionary
df['ZIP_CODE_ENCODED'] = df['ZIP_CODE'].map(zip_code_to_mean)


In [14]:
scaler = MinMaxScaler()
df[['LATITUDE', 'LONGITUDE']] = scaler.fit_transform(df[['LATITUDE', 'LONGITUDE']])


In [15]:
df_ml = df.drop(columns=['ZIP_CODE', 'INSPECTION_DATE', 'RESULT'])

In [16]:
# One-hot encode the BOROUGH columns
df_ml = pd.get_dummies(df_ml, columns=['BOROUGH', 'INSPECTION_TYPE'], drop_first=True)

# Display the resulting dataframe
df_ml.head()

,LATITUDE,LONGITUDE,RAT_ACTIVITY,ZIP_CODE_ENCODED,BOROUGH_Brooklyn,BOROUGH_Manhattan,BOROUGH_Queens,BOROUGH_Staten Island,INSPECTION_TYPE_CLEAN_UPS,INSPECTION_TYPE_Compliance,INSPECTION_TYPE_Initial,INSPECTION_TYPE_STOPPAGE
0,0.771314,0.568308,0,0.333333,0,1,0,0,0,0,1,0
1,0.578179,0.634004,0,0.583062,0,0,1,0,0,0,1,0
2,0.850120,0.656294,0,0.421042,0,0,0,0,0,0,1,0
3,0.505417,0.609532,1,0.632289,0,0,1,0,0,0,0,0
4,0.965257,0.714036,0,0.053110,0,0,0,0,0,0,1,0


In [17]:
# Split our preprocessed data into our features and target arrays
y = df_ml["RAT_ACTIVITY"].values
X = df_ml.drop(columns=["RAT_ACTIVITY"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
print(X_train_scaled.shape)

(121782, 11)


In [20]:
#this is in the final
from keras_tuner.tuners import Hyperband
from tensorflow.keras.callbacks import ModelCheckpoint

In [22]:
#This is in your final

def build_model(hp):
    model = tf.keras.models.Sequential()

    # First hidden layer
    model.add(tf.keras.layers.Dense(units=hp.Int('units_layer1', min_value=32, max_value=256, step=32), 
                                    activation="relu", 
                                    input_dim=X_train_scaled.shape[1]))

    # Second hidden layer
    model.add(tf.keras.layers.Dense(units=hp.Int('units_layer2', min_value=16, max_value=128, step=16), 
                                    activation="relu"))

    # Output layer
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Check the structure of the model
    model.summary()

    # Compile the model with a searchable learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                384       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 929
Trainable params: 929
Non-trainable params: 0
_________________________________________________________________


In [ ]:

tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=200,
    directory='results/nyc_dataset8',
    project_name='rat_optimizer'
)

In [23]:
tuner.search(X_train_scaled, y_train, epochs=150, validation_data=(X_test_scaled, y_test))

Trial 73 Complete [00h 00m 19s]
val_accuracy: 0.8010838627815247

Best val_accuracy So Far: 0.8014041185379028
Total elapsed time: 00h 18m 24s

Search: Running Trial #74

Value             |Best Value So Far |Hyperparameter
256               |96                |units_layer1
128               |48                |units_layer2
0.01              |0.001             |learning_rate
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               3072      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                            

KeyboardInterrupt: 

Trial 209 Complete [00h 29m 16s]
val_accuracy: 0.8078088164329529

Best val_accuracy So Far: 0.8078827261924744
Total elapsed time: 03h 46m 45s

Search: Running Trial #210

Value             |Best Value So Far |Hyperparameter
64                |256               |units_layer1
112               |112               |units_layer2
0.001             |0.001             |learning_rate
200               |200               |tuner/epochs
67                |67                |tuner/initial_epoch
3                 |4                 |tuner/bracket
3                 |4                 |tuner/round
0203              |0142              |tuner/trial_id

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_model = build_model(best_hyperparameters)

In [ ]:
# Compile the model (you can directly use the values from the best hyperparameters)
best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_hyperparameters.get('learning_rate')),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

# Train your Model with the Best Hyperparameters
# Create an EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Train the model
history = best_model.fit(X_train_scaled,
                         y_train,
                         epochs=100,
                         batch_size=32,
                         validation_data=(X_test_scaled, y_test),
                         callbacks=[early_stopping])


In [ ]:
# Evaluate the model
loss, accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

In [ ]:
# Save the best_model
best_model.save('model/my_rat_model')

In [ ]:
# Generate predictions on the test set
predictions = best_model.predict(X_test_scaled)

# If you want binary predictions (0 or 1) since you're using binary_crossentropy
binary_predictions = (predictions > 0.5).astype("int32")

That was the test of the nn on the full data set. Now we will train it on the boroughs to see how it does there. We will look at Brooklyn, Bronx and Manhattan (highest incidences of rats)

In [ ]:
# Filter the DataFrame into separate DataFrames for each borough
staten_island = df[df['BOROUGH'] == 'Staten Island']
bronx = df[df['BOROUGH'] == 'Bronx']
brooklyn = df[df['BOROUGH'] == 'Brooklyn']
manhattan = df[df['BOROUGH'] == 'Manhattan']
queens = df[df['BOROUGH'] == 'Queens']


In [ ]:
def preprocess_borough_data(borough_data):
    # Drop the unnecessary columns
    borough_data.drop(columns=['INSPECTION_DATE', 'RESULT', 'BOROUGH'], inplace=True)

    # One-hot encode the INSPECTION_TYPE column
    borough_data = pd.get_dummies(borough_data, columns=['INSPECTION_TYPE'], drop_first=True)

    # Drop the ZIP_CODE column (since we already have the encoded version)
    borough_data.drop(columns=['ZIP_CODE'], inplace=True)
    
    return borough_data

# Preprocess the data for each borough
staten_island_ml = preprocess_borough_data(staten_island)
bronx_ml = preprocess_borough_data(bronx)
brooklyn_ml = preprocess_borough_data(brooklyn)
manhattan_ml = preprocess_borough_data(manhattan)
queens_ml = preprocess_borough_data(queens)

BROOKLYN DATA TEST

In [ ]:
# Split our preprocessed data into our features and target arrays
y = brooklyn_ml["RAT_ACTIVITY"].values
X = brooklyn_ml.drop(columns=["RAT_ACTIVITY"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print(X_train_scaled.shape)

In [ ]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

# Train the model
history = nn.fit(
    X_train_scaled,
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_scaled, y_test)
)

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

BRONX DATA TEST

In [ ]:
# Split our preprocessed data into our features and target arrays
y = bronx_ml["RAT_ACTIVITY"].values
X = bronx_ml.drop(columns=["RAT_ACTIVITY"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print(X_train_scaled.shape)

In [ ]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

# Train the model
history = nn.fit(
    X_train_scaled,
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_scaled, y_test)
)

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

MANHATTAN DATA TEST

In [ ]:
# Split our preprocessed data into our features and target arrays
y = manhattan_ml["RAT_ACTIVITY"].values
X = manhattan_ml.drop(columns=["RAT_ACTIVITY"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print(X_train_scaled.shape)

In [ ]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

# Train the model
history = nn.fit(
    X_train_scaled,
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_scaled, y_test)
)

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Testing Notes:

Base test: 16/8/1/50 epochs
Full Dataset: 1292/1292 - 1s - loss: 0.4138 - accuracy: 0.8003 - 858ms/epoch - 664us/step
Loss: 0.4137950539588928, Accuracy: 0.8003047704696655

Brooklyn: 385/385 - 0s - loss: 0.4390 - accuracy: 0.7908 - 273ms/epoch - 708us/step
Loss: 0.4390130043029785, Accuracy: 0.7908262610435486

Bronx: 374/374 - 0s - loss: 0.4202 - accuracy: 0.7940 - 264ms/epoch - 707us/step
Loss: 0.4202301502227783, Accuracy: 0.7940191030502319

Manhattan: 429/429 - 0s - loss: 0.3873 - accuracy: 0.8155 - 338ms/epoch - 788us/step
Loss: 0.38726359605789185, Accuracy: 0.8155113458633423

First Test: 32/16/1/100 epochs


Per the NYC open data:

"Initial Inspection - Inspection conducted in response to a 311 complaint, or a proactive inspection conducted through our neighborhood indexing program" and "Compliance Inspection - If a property fails its initial inspection, the Health Department will conduct a follow up (Compliance) inspection"

In [ ]:
# Create a dictionary to store the total initial inspections for each borough
borough_totals = {
    'Staten Island': len(staten_island[staten_island['INSPECTION_TYPE'] == 'Initial']),
    'Bronx': len(bronx[bronx['INSPECTION_TYPE'] == 'Initial']),
    'Brooklyn': len(brooklyn[brooklyn['INSPECTION_TYPE'] == 'Initial']),
    'Manhattan': len(manhattan[manhattan['INSPECTION_TYPE'] == 'Initial']),
    'Queens': len(queens[queens['INSPECTION_TYPE'] == 'Initial'])
}

# Convert the dictionary to a Pandas Series for easy plotting
borough_series = pd.Series(borough_totals)

# Create a bar chart
borough_series.plot(kind='bar', stacked=True, figsize=(10, 6))

# Customize the plot
plt.title('Total Initial Inspections by Borough')
plt.xlabel('Borough')
plt.ylabel('Total Initial Inspections')

# Show the plot
plt.savefig("./Graphics/total-initial-inspections-by-borough.png")
plt.show()

In [ ]:
# Create a dictionary to store the total compliance inspections for each borough
borough_compliance = {
    'Staten Island': len(staten_island[staten_island['INSPECTION_TYPE'] == 'Compliance']),
    'Bronx': len(bronx[bronx['INSPECTION_TYPE'] == 'Compliance']),
    'Brooklyn': len(brooklyn[brooklyn['INSPECTION_TYPE'] == 'Compliance']),
    'Manhattan': len(manhattan[manhattan['INSPECTION_TYPE'] == 'Compliance']),
    'Queens': len(queens[queens['INSPECTION_TYPE'] == 'Compliance'])
}

# Convert the dictionary to a Pandas Series for easy plotting
borough_comp_series = pd.Series(borough_compliance)

# Create a bar chart
borough_comp_series.plot(kind='bar', stacked=True, figsize=(10, 6))

# Customize the plot
plt.title('Total Compliance Inspections by Borough')
plt.xlabel('Borough')
plt.ylabel('Total Compliance Inspections')

# Show the plot
plt.savefig("./Graphics/total-compliance-inspections-by-borough.png")
plt.show()

In [ ]:
# Create a DataFrame to store the total initial and compliance inspections for each borough
borough_data = pd.DataFrame({
    'Borough': ['Staten Island', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens'],
    'Initial': [
        len(staten_island[staten_island['INSPECTION_TYPE'] == 'Initial']),
        len(bronx[bronx['INSPECTION_TYPE'] == 'Initial']),
        len(brooklyn[brooklyn['INSPECTION_TYPE'] == 'Initial']),
        len(manhattan[manhattan['INSPECTION_TYPE'] == 'Initial']),
        len(queens[queens['INSPECTION_TYPE'] == 'Initial'])
    ],
    'Compliance': [
        len(staten_island[staten_island['INSPECTION_TYPE'] == 'Compliance']),
        len(bronx[bronx['INSPECTION_TYPE'] == 'Compliance']),
        len(brooklyn[brooklyn['INSPECTION_TYPE'] == 'Compliance']),
        len(manhattan[manhattan['INSPECTION_TYPE'] == 'Compliance']),
        len(queens[queens['INSPECTION_TYPE'] == 'Compliance'])
    ]
})

# Create a stacked bar chart
borough_data.set_index('Borough').plot(kind='bar', stacked=True, figsize=(10, 6))

# Customize the plot
plt.title('Total Initial and Compliance Inspections by Borough')
plt.xlabel('Borough')
plt.ylabel('Total Inspections')

# Save the plot
plt.savefig("./Graphics/total-inspections-by-Borough.png")

# Show the plot
plt.show()

In [ ]:
# Display all unique values in the 'result' column for Brooklyn
unique_results = df['RESULT'].unique()

print("Unique Result Values:")
for result in unique_results:
    print(result)

In [ ]:
# Display all unique values in the 'result' column for Brooklyn
unique_inspections = df['INSPECTION_TYPE'].unique()

print("Unique inspection type Values:")
for result in unique_inspections:
    print(result)

Compliance visits are only triggered after the initial visit (if rat evidence is indicated). Similarly, for results: I think everything except Passed, failed for other R, and possibly clean up are indicative of rat activity. According to NYC Open data, here are the inspection categories:
Baiting - Application of rodenticide, or monitoring visit by a
Health Department Pest Control Professional.
Clean Up - The removal of garbage and clutter from a
property by the Health Department.
Stoppage – Sealing of holes and cracks allowing free
movement of pests.
Initial Inspection - Inspection conducted in response to a 311
complaint, or a proactive inspection conducted through our
neighborhood indexing program.
Compliance Inspection - If a property fails its initial
inspection, the Health Department will conduct a follow up
(Compliance) inspection.
Baiting - Application of rodenticide, or monitoring visit by a
Health Department Pest Control Professional.
Clean Up - The removal of garbage and clutter from a
property by the Health Department.

Filter Relevant Data: If we're interested in tracking the progression of rat activity and control measures, we should filter our data to only include records where an initial inspection resulted in a finding of rat activity.

In [ ]:
#FEATURE ENGINEERING
brooklyn_df = brooklyn.copy()

brooklyn_df = brooklyn_df.drop(columns=['_id', 'JOB_TICKET_OR_WORK_ORDER_ID', 'JOB_PROGRESS', 'BOROUGH'])

In [ ]:
brooklyn_df

In [ ]:
job_id_counts = brooklyn_df['JOB_ID'].value_counts()

# Filter to only show job_ids that appear more than once
repeated_job_ids = job_id_counts[job_id_counts > 40]

print("Job_IDs that repeat and their frequencies:")
print(repeated_job_ids)


In [ ]:
# List of job IDs to filter
job_ids = ['PC7775029', 'PC7860266', 'PC7870102', 'PC7845837', 'PC7870111', 'PC7868958', 'PC7881156', 'PC7931761', 'PC7930725', 'PC7931755' ]

# Create an empty dictionary to store the filtered dataframes
filtered_dfs = {}

# Loop through the job IDs and create filtered dataframes for each
for job_id in job_ids:
    job_id_filter = brooklyn_df['JOB_ID'] == job_id
    filtered_df = brooklyn_df[job_id_filter]
    
    # Sort the filtered dataframe by the INSPECTION_DATE column
    sorted_filtered_df = filtered_df.sort_values(by='INSPECTION_DATE', ascending=True)
    
    filtered_dfs[job_id] = sorted_filtered_df

# Access the dataframe for a specific job ID
print("Dataframe for job ID PC7775029:")
filtered_dfs['PC7775029']

In [ ]:
# Create an empty dictionary to store the count of "passed" results for each job ID
passed_counts = {}

# Loop through the filtered dataframes and count the number of "passed" results for each job ID
for job_id, df in filtered_dfs.items():
    passed_filter = df['RESULT'] == 'Passed'
    passed_counts[job_id] = df[passed_filter].shape[0]

# Print the count of "passed" results for each job ID
print("Count of 'Passed' results for each job ID:")
print(passed_counts)

Use high frequency job_ids to possibly look at the following:

Timeline of Inspections: Analyze the time gap between inspections (Initial and Compliance) within a specific job ID. This could help you understand how frequently inspections are conducted, and how long it typically takes for a property to move from the initial inspection to the compliance inspection.

Inspection Outcomes: Check the sequence of inspection outcomes for each job ID. Did the property initially fail the inspection, then pass subsequent inspections? Or did it continue to fail even after multiple inspections? This can provide insights into treatment effectiveness.

Treatment Types: Analyze the types of treatments (Baiting, Cleanup, Stoppage) applied within each job ID. Did the property receive multiple treatments of the same type, or a combination of different treatments?

Changes in Rat Activity: Track changes in rat activity (if available) within each job ID, such as the reduction or increase in sightings, based on inspection results and associated timestamps.

Location Data: Look at the geographic location of the properties associated with each job ID. Are there clusters of properties with high-frequency job IDs in specific areas? This could indicate localized infestations.




Look at zip code data vs inspections: rate of growth in zip vs rate of treatment/rate of inspection
look at full data set (2023), specifically what zip codes have a high rate of "rat activity", predict "Treatment zones" based on "rat activity" and inspection results
